## Train VGG11 frist for initialization

In [ ]:
from model import vgg
from script.function import train, test, save
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchmetrics
import time
import datetime
import os
import numpy as np

# Set random_seed and environment parameters
#################################################################
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.manual_seed(1234)

# Get cpu or gpu device for training.
#################################################################
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Initialize model
#################################################################
# weight_path = './weights/vgg19.pt'
image_size = 224
classes_num = 15
model = vgg.NeuralNetwork(type = 'A', classes = classes_num).to(device)
# state_dict = torch.load(weight_path,map_location=device)
# model.load_state_dict(state_dict, strict = False)

# Set hyper-parameters
#################################################################
# opt = torch.optim.RMSprop(model.parameters(), lr = 1e-4)
lr = 1e-2
opt = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 5e-4)
loss_func = nn.functional.cross_entropy
metric_func = lambda x,y:torchmetrics.functional.accuracy(x, y, task="multiclass", num_classes=classes_num)
metric_name = "accuracy"

# Construct dataset
#################################################################
transforms_train = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomGrayscale(p=0.1),
#     transforms.GaussianBlur([3, 3], sigma=(0.1, 4.0)),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

transforms_val = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
])

# base_dir = './imagenet-object-localization-challenge/ILSVRC/Data/ILSVRC/'
base_dir = './data/15classes/'
train_path = base_dir+'train/'
val_path = base_dir+'test/'
batch_size = 64
train_data = datasets.ImageFolder(train_path, transform=transforms_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=6, prefetch_factor = 5)

val_data = datasets.ImageFolder(val_path, transform=transforms_val)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True)

# Start to train model
#################################################################
epochs = 200
acc=[]
val_acc=[]
loss=[]
val_loss=[]
count = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------------")
    start = time.time()
    a,b = train(train_loader, model, loss_func, metric_func, opt, device)
    end = time.time()
    print(str(datetime.timedelta(seconds=int(round((end - start))))))
    acc.append(a)
    loss.append(b)
    start = time.time()
    a,b = test(val_loader, model, loss_func, metric_func, device)
    end = time.time()
    print(str(datetime.timedelta(seconds=int(round((end - start))))))
    val_acc.append(a)
    val_loss.append(b)
    print(f"------------------------------------\n")
    if count < 10:
        count += 1
        continue
    if np.mean(acc[t-9:t+1]) > 0.9 and np.mean(acc[t-4:t+1]) - np.mean(acc[t-9:t-4]) < 0.001:
        break
    if np.mean(val_acc[t-4:t+1]) <= np.mean(val_acc[t-9:t-4]):
        lr /= 10
        opt = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 5e-4)
        print('Learning rate adjusted to', lr)
        count = 0
    save(model,'./weights/vgg11_checkpoint.pt')
print("Done!")

# Plot
#################################################################
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
save(model,'./weights/vgg11_full.pt')
del model
torch.cuda.empty_cache()
time.sleep(5)

## Initialize VGG19 using pre-trained VGG11's weights

In [ ]:
from model import vgg
from script.function import train, test, save
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchmetrics
import time
import datetime
import os
import numpy as np

# Set random_seed and environment parameters
#################################################################
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
torch.manual_seed(1234)

# Get cpu or gpu device for training.
#################################################################
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Initialize model
#################################################################
weight_path = './weights/vgg11_full.pt'
image_size = 224
classes_num = 15
model = vgg.NeuralNetwork(type = 'E', classes = classes_num).to(device)
state_dict = torch.load(weight_path,map_location=device)
model.load_state_dict(state_dict, strict = False)

# Set hyper-parameters
#################################################################
# opt = torch.optim.RMSprop(model.parameters(), lr = 1e-4)
lr = 1e-2
opt = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 5e-4)
loss_func = nn.functional.cross_entropy
metric_func = lambda x,y:torchmetrics.functional.accuracy(x, y, task="multiclass", num_classes=classes_num)
metric_name = "accuracy"

# Construct dataset
#################################################################
transforms_train = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomGrayscale(p=0.1),
#     transforms.GaussianBlur([3, 3], sigma=(0.1, 3.0)),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

transforms_val = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
])

base_dir = './data/15classes/'
train_path = base_dir+'train/'
val_path = base_dir+'test/'
batch_size = 32
train_data = datasets.ImageFolder(train_path, transform=transforms_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=6, prefetch_factor = 5)

val_data = datasets.ImageFolder(val_path, transform=transforms_val)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last=True)

# Start to train model
#################################################################
epochs = 300
acc=[]
val_acc=[]
loss=[]
val_loss=[]
count = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------------")
    start = time.time()
    a,b = train(train_loader, model, loss_func, metric_func, opt, device)
    end = time.time()
    print(str(datetime.timedelta(seconds=int(round((end - start))))))
    acc.append(a)
    loss.append(b)
    start = time.time()
    a,b = test(val_loader, model, loss_func, metric_func, device)
    end = time.time()
    print(str(datetime.timedelta(seconds=int(round((end - start))))))
    val_acc.append(a)
    val_loss.append(b)
    print(f"------------------------------------\n")
    if count <= 10:
        count += 1
        continue
    if np.mean(acc[t-9:t+1]) > 0.9 and np.mean(acc[t-4:t+1]) - np.mean(acc[t-9:t-4]) < 0.001:
        break
    if np.mean(val_acc[t-4:t+1]) <= np.mean(val_acc[t-9:t-4]):
        lr /= 10
        opt = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 5e-4)
        print('Learning rate adjusted to', lr)
        count = 0
    save(model,'./weights/vgg19_checkpoint.pt')
print("Done!")

# Plot
#################################################################
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
save(model,'./weights/vgg19_full.pt')
del model
torch.cuda.empty_cache()
time.sleep(5)